In [139]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import warnings
warnings.filterwarnings('ignore')

In [140]:
df=pd.read_csv('../data_col/FinalDataSetMonths.csv', index_col=0)
df

,Name,Platform,Year,Genre,Publisher,DateSortie,Studio,Score,Rate,Review,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,GK_licence,GK_distributeur,Mois
0,Wii Sports,Wii,2006.0,Sports,Nintendo,2006-11-01,Nintendo,7.6,7.05,210.0,41.49,29.02,3.77,8.46,82.74,Wii Sports,Nintendo,11.0
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,0000-01-00,Nintendo,7.0,8.40,235.0,29.08,3.58,6.81,0.77,40.24,Mario,NaN,1.0
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,2008-04-01,Nintendo,7.6,8.00,319.0,15.85,12.88,3.79,3.31,35.82,Mario,NaN,4.0
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,2009-07-01,Nintendo,7.5,7.35,103.0,15.75,11.01,3.28,2.96,33.00,Wii Sports,NaN,7.0
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,0000-01-00,Nintendo,7.0,8.20,138.0,11.27,8.89,10.22,1.00,31.37,PokǸmon,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17888,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0000-01-00,NaN,NaN,NaN,NaN,0.01,0.00,0.00,0.00,0.01,Woody Woodpecker,Nobilis,1.0
17889,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,2003-02-01,Atari,3.0,NaN,NaN,0.01,0.00,0.00,0.00,0.01,Men In Black,NaN,2.0
17890,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0000-01-00,Left Field Productions,2.0,NaN,NaN,0.00,0.00,0.00,0.00,0.01,NaN,Activision,1.0
17891,Know How 2,DS,2010.0,Puzzle,7G//AMES,0000-01-00,InXile Entertainment,8.0,7.30,38.0,0.00,0.01,0.00,0.00,0.01,Wasteland,Deep Silver,1.0


In [141]:
df['Year2'] = df['DateSortie'].apply(lambda x: x[:4])
df['Year2'] = df['Year2'].astype(float)
df['Year'] = df['Year'].fillna(df['Year2'])
df
df = df.drop(['Year2', 'DateSortie'], axis=1)
df['Year'] = df['Year'].replace(0, np.nan)
df = df.dropna(axis = 0, how = 'all', subset = ['Year'])
df['Year'] = df['Year'].astype(int).astype(str)
df['Mois'] = df['Mois'].astype(int).astype(str)
for i,row in df.iterrows():
    if len(df.loc[i, 'Mois']) == 1:
        df.loc[i, 'Date_Sortie'] = df.loc[i,'Year']+'-0'+df.loc[i,'Mois']+'-01'
    else:
        df.loc[i, 'Date_Sortie'] = df.loc[i,'Year']+'-'+df.loc[i,'Mois']+'-01'
df['Date_Sortie'] = pd.to_datetime(df['Date_Sortie'])

In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16538 entries, 0 to 17892
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Name             16538 non-null  object        
 1   Platform         16538 non-null  object        
 2   Year             16538 non-null  object        
 3   Genre            16538 non-null  object        
 4   Publisher        16495 non-null  object        
 5   Studio           13917 non-null  object        
 6   Score            10441 non-null  float64       
 7   Rate             11556 non-null  float64       
 8   Review           9928 non-null   float64       
 9   NA_Sales         16538 non-null  float64       
 10  EU_Sales         16538 non-null  float64       
 11  JP_Sales         16538 non-null  float64       
 12  Other_Sales      16538 non-null  float64       
 13  Global_Sales     16538 non-null  float64       
 14  GK_licence       10863 non-null  objec

# ROLLING MEANS AVEC LES SCORE ET RATE

In [143]:
# publisher

df_publisher = df.sort_values(by = ['Publisher','Date_Sortie'], ascending=True)
publishers = df_publisher['Publisher'].unique()
publishers = np.delete(publishers, -1)
df_publisher['RM_Publisher'] = np.nan
df_publisher["RM_Publisher_score"] = np.nan
df_publisher["RM_Publisher_rate"] = np.nan
df_publisher["RM_Publisher_reviews"] = np.nan
df_rm1on3 = pd.DataFrame(columns=df_publisher.columns)
for publisher in publishers:
    df_temp = df_publisher[df_publisher['Publisher']==publisher]
    nb_games = df_temp.shape[0]
    if nb_games == 1:
        df_temp.iloc[0,-4] = 0
        df_temp.iloc[0,-3] = 0
        df_temp.iloc[0,-2] = 0
        df_temp.iloc[0,-1] = 0
    elif nb_games == 2:
        df_temp.iloc[0,-4] = 0
        df_temp.iloc[0,-3] = 0
        df_temp.iloc[0,-2] = 0
        df_temp.iloc[0,-1] = 0
        df_temp.iloc[1,-4] = df_temp.iloc[0,13]
        df_temp.iloc[1,-3] = df_temp.iloc[0,6]
        df_temp.iloc[1,-2] = df_temp.iloc[0,7]
        df_temp.iloc[1,-1] = df_temp.iloc[0,8]
    elif nb_games == 3:
        df_temp.iloc[0,-4] = 0
        df_temp.iloc[0,-3] = 0
        df_temp.iloc[0,-2] = 0
        df_temp.iloc[0,-1] = 0
        df_temp.iloc[1,-4] = df_temp.iloc[0,13]
        df_temp.iloc[1,-3] = df_temp.iloc[0,6]
        df_temp.iloc[1,-2] = df_temp.iloc[0,7]
        df_temp.iloc[1,-1] = df_temp.iloc[0,8]
        df_temp.iloc[2,-4] = (df_temp.iloc[0:1,13]).mean()
        df_temp.iloc[2,-3] = (df_temp.iloc[0:1,6]).mean()
        df_temp.iloc[2,-2] = (df_temp.iloc[0:1,7]).mean()
        df_temp.iloc[2,-1] = (df_temp.iloc[0:1,8]).mean()
    else:
        df_temp.iloc[0,-4] = 0
        df_temp.iloc[0,-3] = 0
        df_temp.iloc[0,-2] = 0
        df_temp.iloc[0,-1] = 0
        df_temp.iloc[1,-4] = df_temp.iloc[0,13]
        df_temp.iloc[1,-3] = df_temp.iloc[0,6]
        df_temp.iloc[1,-2] = df_temp.iloc[0,7]
        df_temp.iloc[1,-1] = df_temp.iloc[0,8]        
        for i in range(2,nb_games):
            df_temp.iloc[i,-4] = (df_temp.iloc[(i-2):(i-1),13]).mean()
            df_temp.iloc[i,-3] = (df_temp.iloc[(i-2):(i-1),6]).mean()
            df_temp.iloc[i,-2] = (df_temp.iloc[(i-2):(i-1),7]).mean()
            df_temp.iloc[i,-1] = (df_temp.iloc[(i-2):(i-1),8]).mean()
    df_rm1on3 = pd.concat([df_rm1on3,df_temp], axis = 0)
df_rm1on3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16495 entries, 16962 to 14861
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Name                  16495 non-null  object        
 1   Platform              16495 non-null  object        
 2   Year                  16495 non-null  object        
 3   Genre                 16495 non-null  object        
 4   Publisher             16495 non-null  object        
 5   Studio                13893 non-null  object        
 6   Score                 10427 non-null  float64       
 7   Rate                  11542 non-null  float64       
 8   Review                9916 non-null   float64       
 9   NA_Sales              16495 non-null  float64       
 10  EU_Sales              16495 non-null  float64       
 11  JP_Sales              16495 non-null  float64       
 12  Other_Sales           16495 non-null  float64       
 13  Global_Sales

In [144]:
df_rm = df.merge(right=df_rm1on3, on=['Name', 'Platform', 'Year', 'Genre', 'Publisher', 'Studio', 'Score',
       'Rate', 'Review', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales',
       'Global_Sales', 'GK_licence', 'GK_distributeur', 'Mois', 'Date_Sortie',], how='left')

In [145]:
df_rm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16538 entries, 0 to 16537
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Name                  16538 non-null  object        
 1   Platform              16538 non-null  object        
 2   Year                  16538 non-null  object        
 3   Genre                 16538 non-null  object        
 4   Publisher             16495 non-null  object        
 5   Studio                13917 non-null  object        
 6   Score                 10441 non-null  float64       
 7   Rate                  11556 non-null  float64       
 8   Review                9928 non-null   float64       
 9   NA_Sales              16538 non-null  float64       
 10  EU_Sales              16538 non-null  float64       
 11  JP_Sales              16538 non-null  float64       
 12  Other_Sales           16538 non-null  float64       
 13  Global_Sales    

In [146]:
# studio

df_studio = df_rm.sort_values(by = ['Studio','Date_Sortie'], ascending=True)
studios = df_studio['Studio'].unique()
studios = np.delete(studios, -1)
df_studio['RM_Studio'] = np.nan
df_studio["RM_Studio_score"] = np.nan
df_studio["RM_Studio_rate"] = np.nan
df_studio["RM_Studio_reviews"] = np.nan
df_rm2on3 = pd.DataFrame(columns=df_studio.columns)
for studio in studios:
    df_temp = df_studio[df_studio['Studio']==studio]
    nb_games = df_temp.shape[0]
    if nb_games == 1:
        df_temp.iloc[0,-4] = df_temp.iloc[0,-8]
        df_temp.iloc[0,-3] = df_temp.iloc[0,-7]
        df_temp.iloc[0,-2] = df_temp.iloc[0,-6]
        df_temp.iloc[0,-1] = df_temp.iloc[0,-5]
    elif nb_games == 2:
        df_temp.iloc[0,-4] = df_temp.iloc[0,-8]
        df_temp.iloc[0,-3] = df_temp.iloc[0,-7]
        df_temp.iloc[0,-2] = df_temp.iloc[0,-6]
        df_temp.iloc[0,-1] = df_temp.iloc[0,-5]
        df_temp.iloc[1,-4] = df_temp.iloc[0,13]
        df_temp.iloc[1,-3] = df_temp.iloc[0,6]
        df_temp.iloc[1,-2] = df_temp.iloc[0,7]
        df_temp.iloc[1,-1] = df_temp.iloc[0,8]
    elif nb_games == 3:
        df_temp.iloc[0,-4] = df_temp.iloc[0,-8]
        df_temp.iloc[0,-3] = df_temp.iloc[0,-7]
        df_temp.iloc[0,-2] = df_temp.iloc[0,-6]
        df_temp.iloc[0,-1] = df_temp.iloc[0,-5]
        df_temp.iloc[1,-4] = df_temp.iloc[0,13]
        df_temp.iloc[1,-3] = df_temp.iloc[0,6]
        df_temp.iloc[1,-2] = df_temp.iloc[0,7]
        df_temp.iloc[1,-1] = df_temp.iloc[0,8]
        df_temp.iloc[2,-4] = (df_temp.iloc[0:1,13]).mean()
        df_temp.iloc[2,-3] = (df_temp.iloc[0:1,6]).mean()
        df_temp.iloc[2,-2] = (df_temp.iloc[0:1,7]).mean()
        df_temp.iloc[2,-1] = (df_temp.iloc[0:1,8]).mean()
    else:
        df_temp.iloc[0,-4] = df_temp.iloc[0,-8]
        df_temp.iloc[0,-3] = df_temp.iloc[0,-7]
        df_temp.iloc[0,-2] = df_temp.iloc[0,-6]
        df_temp.iloc[0,-1] = df_temp.iloc[0,-5]
        df_temp.iloc[1,-4] = df_temp.iloc[0,13]
        df_temp.iloc[1,-3] = df_temp.iloc[0,6]
        df_temp.iloc[1,-2] = df_temp.iloc[0,7]
        df_temp.iloc[1,-1] = df_temp.iloc[0,8]        
        for i in range(2,nb_games):
            df_temp.iloc[i,-4] = (df_temp.iloc[(i-2):(i-1),13]).mean()
            df_temp.iloc[i,-3] = (df_temp.iloc[(i-2):(i-1),6]).mean()
            df_temp.iloc[i,-2] = (df_temp.iloc[(i-2):(i-1),7]).mean()
            df_temp.iloc[i,-1] = (df_temp.iloc[(i-2):(i-1),8]).mean()
    df_rm2on3 = pd.concat([df_rm2on3,df_temp], axis = 0)
df_rm2on3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13917 entries, 8613 to 15829
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Name                  13917 non-null  object        
 1   Platform              13917 non-null  object        
 2   Year                  13917 non-null  object        
 3   Genre                 13917 non-null  object        
 4   Publisher             13893 non-null  object        
 5   Studio                13917 non-null  object        
 6   Score                 10146 non-null  float64       
 7   Rate                  11109 non-null  float64       
 8   Review                9473 non-null   float64       
 9   NA_Sales              13917 non-null  float64       
 10  EU_Sales              13917 non-null  float64       
 11  JP_Sales              13917 non-null  float64       
 12  Other_Sales           13917 non-null  float64       
 13  Global_Sales 

In [147]:
df_rm = df_rm.merge(right=df_rm2on3, on=['Name', 'Platform', 'Year', 'Genre', 'Publisher', 'Studio', 'Score',
       'Rate', 'Review', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales',
       'Global_Sales', 'GK_licence', 'GK_distributeur', 'Mois', 'Date_Sortie',
        'RM_Publisher', 'RM_Publisher_score', 'RM_Publisher_rate', 'RM_Publisher_reviews'], how='left')

In [148]:
df_rm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16538 entries, 0 to 16537
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Name                  16538 non-null  object        
 1   Platform              16538 non-null  object        
 2   Year                  16538 non-null  object        
 3   Genre                 16538 non-null  object        
 4   Publisher             16495 non-null  object        
 5   Studio                13917 non-null  object        
 6   Score                 10441 non-null  float64       
 7   Rate                  11556 non-null  float64       
 8   Review                9928 non-null   float64       
 9   NA_Sales              16538 non-null  float64       
 10  EU_Sales              16538 non-null  float64       
 11  JP_Sales              16538 non-null  float64       
 12  Other_Sales           16538 non-null  float64       
 13  Global_Sales    

In [149]:
df_rm["RM_Studio"]=df_rm["RM_Studio"].fillna(df_rm["RM_Publisher"])
df_rm["RM_Studio_score"]=df_rm["RM_Studio_score"].fillna(df_rm["RM_Publisher_score"])
df_rm["RM_Studio_rate"]=df_rm["RM_Studio_rate"].fillna(df_rm["RM_Publisher_rate"])
df_rm["RM_Studio_reviews"]=df_rm["RM_Studio_reviews"].fillna(df_rm["RM_Publisher_reviews"])

In [150]:
df_rm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16538 entries, 0 to 16537
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Name                  16538 non-null  object        
 1   Platform              16538 non-null  object        
 2   Year                  16538 non-null  object        
 3   Genre                 16538 non-null  object        
 4   Publisher             16495 non-null  object        
 5   Studio                13917 non-null  object        
 6   Score                 10441 non-null  float64       
 7   Rate                  11556 non-null  float64       
 8   Review                9928 non-null   float64       
 9   NA_Sales              16538 non-null  float64       
 10  EU_Sales              16538 non-null  float64       
 11  JP_Sales              16538 non-null  float64       
 12  Other_Sales           16538 non-null  float64       
 13  Global_Sales    

In [151]:
# licence

df_licence = df_rm.sort_values(by = ['GK_licence','Date_Sortie'], ascending=True)
licences = df_licence['GK_licence'].unique()
licences = np.delete(licences, -1)
df_licence['RM_Licence'] = np.nan
df_licence["RM_Licence_score"] = np.nan
df_licence["RM_Licence_rate"] = np.nan
df_licence["RM_Licence_reviews"] = np.nan
df_rm3on3 = pd.DataFrame(columns=df_licence.columns)
for licence in licences:
    df_temp = df_licence[df_licence['GK_licence']==licence]
    nb_games = df_temp.shape[0]
    if nb_games == 1:
        df_temp.iloc[0,-4] = df_temp.iloc[0,-8]
        df_temp.iloc[0,-3] = df_temp.iloc[0,-7]
        df_temp.iloc[0,-2] = df_temp.iloc[0,-6]
        df_temp.iloc[0,-1] = df_temp.iloc[0,-5]
    elif nb_games == 2:
        df_temp.iloc[0,-4] = df_temp.iloc[0,-8]
        df_temp.iloc[0,-3] = df_temp.iloc[0,-7]
        df_temp.iloc[0,-2] = df_temp.iloc[0,-6]
        df_temp.iloc[0,-1] = df_temp.iloc[0,-5]
        df_temp.iloc[1,-4] = df_temp.iloc[0,13]
        df_temp.iloc[1,-3] = df_temp.iloc[0,6]
        df_temp.iloc[1,-2] = df_temp.iloc[0,7]
        df_temp.iloc[1,-1] = df_temp.iloc[0,8]
    elif nb_games == 3:
        df_temp.iloc[0,-4] = df_temp.iloc[0,-8]
        df_temp.iloc[0,-3] = df_temp.iloc[0,-7]
        df_temp.iloc[0,-2] = df_temp.iloc[0,-6]
        df_temp.iloc[0,-1] = df_temp.iloc[0,-5]
        df_temp.iloc[1,-4] = df_temp.iloc[0,13]
        df_temp.iloc[1,-3] = df_temp.iloc[0,6]
        df_temp.iloc[1,-2] = df_temp.iloc[0,7]
        df_temp.iloc[1,-1] = df_temp.iloc[0,8]
        df_temp.iloc[2,-4] = (df_temp.iloc[0:1,13]).mean()
        df_temp.iloc[2,-3] = (df_temp.iloc[0:1,6]).mean()
        df_temp.iloc[2,-2] = (df_temp.iloc[0:1,7]).mean()
        df_temp.iloc[2,-1] = (df_temp.iloc[0:1,8]).mean()
    else:
        df_temp.iloc[0,-4] = df_temp.iloc[0,-8]
        df_temp.iloc[0,-3] = df_temp.iloc[0,-7]
        df_temp.iloc[0,-2] = df_temp.iloc[0,-6]
        df_temp.iloc[0,-1] = df_temp.iloc[0,-5]
        df_temp.iloc[1,-4] = df_temp.iloc[0,13]
        df_temp.iloc[1,-3] = df_temp.iloc[0,6]
        df_temp.iloc[1,-2] = df_temp.iloc[0,7]
        df_temp.iloc[1,-1] = df_temp.iloc[0,8]        
        for i in range(2,nb_games):
            df_temp.iloc[i,-4] = (df_temp.iloc[(i-2):(i-1),13]).mean()
            df_temp.iloc[i,-3] = (df_temp.iloc[(i-2):(i-1),6]).mean()
            df_temp.iloc[i,-2] = (df_temp.iloc[(i-2):(i-1),7]).mean()
            df_temp.iloc[i,-1] = (df_temp.iloc[(i-2):(i-1),8]).mean()
    df_rm3on3 = pd.concat([df_rm3on3,df_temp], axis = 0)
df_rm3on3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10863 entries, 9107 to 15058
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Name                  10863 non-null  object        
 1   Platform              10863 non-null  object        
 2   Year                  10863 non-null  object        
 3   Genre                 10863 non-null  object        
 4   Publisher             10842 non-null  object        
 5   Studio                9691 non-null   object        
 6   Score                 7453 non-null   float64       
 7   Rate                  8380 non-null   float64       
 8   Review                7507 non-null   float64       
 9   NA_Sales              10863 non-null  float64       
 10  EU_Sales              10863 non-null  float64       
 11  JP_Sales              10863 non-null  float64       
 12  Other_Sales           10863 non-null  float64       
 13  Global_Sales 

In [152]:
df_rm = df_rm.merge(right=df_rm3on3, on=['Name', 'Platform', 'Year', 'Genre', 'Publisher', 'Studio', 'Score',
       'Rate', 'Review', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales',
       'Global_Sales', 'GK_licence', 'GK_distributeur', 'Mois', 'Date_Sortie',
        'RM_Publisher', 'RM_Publisher_score', 'RM_Publisher_rate', 'RM_Publisher_reviews',
        'RM_Studio', 'RM_Studio_score', 'RM_Studio_rate', 'RM_Studio_reviews'], how='left')
df_rm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16538 entries, 0 to 16537
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Name                  16538 non-null  object        
 1   Platform              16538 non-null  object        
 2   Year                  16538 non-null  object        
 3   Genre                 16538 non-null  object        
 4   Publisher             16495 non-null  object        
 5   Studio                13917 non-null  object        
 6   Score                 10441 non-null  float64       
 7   Rate                  11556 non-null  float64       
 8   Review                9928 non-null   float64       
 9   NA_Sales              16538 non-null  float64       
 10  EU_Sales              16538 non-null  float64       
 11  JP_Sales              16538 non-null  float64       
 12  Other_Sales           16538 non-null  float64       
 13  Global_Sales    

In [153]:
df_rm["RM_Licence"]=df_rm["RM_Licence"].fillna(df_rm["RM_Studio"])
df_rm["RM_Licence_score"]=df_rm["RM_Licence_score"].fillna(df_rm["RM_Studio_score"])
df_rm["RM_Licence_rate"]=df_rm["RM_Licence_rate"].fillna(df_rm["RM_Studio_rate"])
df_rm["RM_Licence_reviews"]=df_rm["RM_Licence_reviews"].fillna(df_rm["RM_Studio_reviews"])
df_rm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16538 entries, 0 to 16537
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Name                  16538 non-null  object        
 1   Platform              16538 non-null  object        
 2   Year                  16538 non-null  object        
 3   Genre                 16538 non-null  object        
 4   Publisher             16495 non-null  object        
 5   Studio                13917 non-null  object        
 6   Score                 10441 non-null  float64       
 7   Rate                  11556 non-null  float64       
 8   Review                9928 non-null   float64       
 9   NA_Sales              16538 non-null  float64       
 10  EU_Sales              16538 non-null  float64       
 11  JP_Sales              16538 non-null  float64       
 12  Other_Sales           16538 non-null  float64       
 13  Global_Sales    

In [154]:
df_rm["RM_Studio"]=df_rm["RM_Studio"].fillna(df_rm["RM_Licence"])
df_rm["RM_Studio_score"]=df_rm["RM_Studio_score"].fillna(df_rm["RM_Licence_score"])
df_rm["RM_Studio_rate"]=df_rm["RM_Studio_rate"].fillna(df_rm["RM_Licence_rate"])
df_rm["RM_Studio_reviews"]=df_rm["RM_Studio_reviews"].fillna(df_rm["RM_Licence_reviews"])

df_rm["RM_Publisher"]=df_rm["RM_Publisher"].fillna(df_rm["RM_Studio"])
df_rm["RM_Publisher_score"]=df_rm["RM_Publisher_score"].fillna(df_rm["RM_Studio_score"])
df_rm["RM_Publisher_rate"]=df_rm["RM_Publisher_rate"].fillna(df_rm["RM_Studio_rate"])
df_rm["RM_Publisher_reviews"]=df_rm["RM_Publisher_reviews"].fillna(df_rm["RM_Studio_reviews"])

df_rm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16538 entries, 0 to 16537
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Name                  16538 non-null  object        
 1   Platform              16538 non-null  object        
 2   Year                  16538 non-null  object        
 3   Genre                 16538 non-null  object        
 4   Publisher             16495 non-null  object        
 5   Studio                13917 non-null  object        
 6   Score                 10441 non-null  float64       
 7   Rate                  11556 non-null  float64       
 8   Review                9928 non-null   float64       
 9   NA_Sales              16538 non-null  float64       
 10  EU_Sales              16538 non-null  float64       
 11  JP_Sales              16538 non-null  float64       
 12  Other_Sales           16538 non-null  float64       
 13  Global_Sales    

In [155]:
df_rm["RM_Publisher_score"]=df_rm["RM_Publisher_score"].fillna(df_rm["RM_Publisher_rate"])
df_rm["RM_Studio_score"]=df_rm["RM_Studio_score"].fillna(df_rm["RM_Studio_rate"])
df_rm["RM_Licence_score"]=df_rm["RM_Licence_score"].fillna(df_rm["RM_Licence_rate"])

df_rm["RM_Publisher_rate"]=df_rm["RM_Publisher_rate"].fillna(df_rm["RM_Publisher_score"])
df_rm["RM_Studio_rate"]=df_rm["RM_Studio_rate"].fillna(df_rm["RM_Studio_score"])
df_rm["RM_Licence_rate"]=df_rm["RM_Licence_rate"].fillna(df_rm["RM_Licence_score"])

df_rm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16538 entries, 0 to 16537
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Name                  16538 non-null  object        
 1   Platform              16538 non-null  object        
 2   Year                  16538 non-null  object        
 3   Genre                 16538 non-null  object        
 4   Publisher             16495 non-null  object        
 5   Studio                13917 non-null  object        
 6   Score                 10441 non-null  float64       
 7   Rate                  11556 non-null  float64       
 8   Review                9928 non-null   float64       
 9   NA_Sales              16538 non-null  float64       
 10  EU_Sales              16538 non-null  float64       
 11  JP_Sales              16538 non-null  float64       
 12  Other_Sales           16538 non-null  float64       
 13  Global_Sales    

In [156]:
df_rm=df_rm.drop(["GK_distributeur","Rate","Score","Review"],axis=1)
df_rm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16538 entries, 0 to 16537
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Name                  16538 non-null  object        
 1   Platform              16538 non-null  object        
 2   Year                  16538 non-null  object        
 3   Genre                 16538 non-null  object        
 4   Publisher             16495 non-null  object        
 5   Studio                13917 non-null  object        
 6   NA_Sales              16538 non-null  float64       
 7   EU_Sales              16538 non-null  float64       
 8   JP_Sales              16538 non-null  float64       
 9   Other_Sales           16538 non-null  float64       
 10  Global_Sales          16538 non-null  float64       
 11  GK_licence            10863 non-null  object        
 12  Mois                  16538 non-null  object        
 13  Date_Sortie     

In [157]:
df_rm["GK_licence"]=df_rm["GK_licence"].fillna(df_rm["Name"])
df_rm['Studio']=df_rm['Studio'].fillna(df_rm['GK_licence'])
df_rm['Publisher']=df_rm['Publisher'].fillna(df_rm['Studio'])
df_rm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16538 entries, 0 to 16537
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Name                  16538 non-null  object        
 1   Platform              16538 non-null  object        
 2   Year                  16538 non-null  object        
 3   Genre                 16538 non-null  object        
 4   Publisher             16538 non-null  object        
 5   Studio                16538 non-null  object        
 6   NA_Sales              16538 non-null  float64       
 7   EU_Sales              16538 non-null  float64       
 8   JP_Sales              16538 non-null  float64       
 9   Other_Sales           16538 non-null  float64       
 10  Global_Sales          16538 non-null  float64       
 11  GK_licence            16538 non-null  object        
 12  Mois                  16538 non-null  object        
 13  Date_Sortie     

# AJOUT D'AUTRES DONNEES PREDICTIVES

In [158]:
# Ajout d'autres données prédictives

##**Est-ce que le jeu fait partie d'une série ?**

def extract_series(url):
    
        names = url.find_all("div",class_="colonnes")
        
        return names
    
url_scrap = "https://fr.wikipedia.org/wiki/Liste_de_s%C3%A9ries_de_jeux_vid%C3%A9o"
url_page = requests.get(url_scrap)
parsed_url_page = BeautifulSoup(url_page.content, 'html.parser')
data_full=extract_series(parsed_url_page)

clean_names=[]
for i in data_full:
    clean_names.append(i.text)
    
clean_names2=[]
for name in clean_names:
    clean_names2.append(name.replace("\n",","))
    
clean_names3=[]
for element in clean_names2:
    clean_names3.append(element[1:-1])

clean_names4=[]
for element in clean_names3:
    clean_names4.append(element.split(","))

fat_list=[]
for sublist in clean_names4:
    for element in sublist:
        fat_list.append(element)


fat_list=pd.Series(fat_list)
fat_list=fat_list.replace(to_replace="Pokémon",value="Pokemon")

chaine = '|'.join(fat_list.values)
r=re.compile(chaine)

masked=[]
for name in df_rm["Name"]:
    if r.match(name):
        masked.append(1)
    else:
        masked.append(0)
        
df_rm["is_serie"]=masked

##**Est-ce que le jeu fait partie d'une des séries les plus vendues ?**

def extract_top_series(url):
    
    li_top_series=[]
    
    names = url.find_all("div",class_="mw-parser-output")
    
    for element in names:
        li=element.find_all("i")
            
    for i in li:
        li_top_series.append(i.text)
    
    return li_top_series
    
url_scrap = "https://fr.wikipedia.org/wiki/Liste_des_s%C3%A9ries_de_jeux_vid%C3%A9o_les_plus_vendues"
url_page = requests.get(url_scrap)
parsed_url_page = BeautifulSoup(url_page.content, 'html.parser')
data_full=extract_top_series(parsed_url_page)

data_full[15]="Pokemon"

data_full=pd.Series(data_full)

chaine = '|'.join(data_full.values)
r=re.compile(chaine)

masked=[]
for name in df_rm["Name"]:
    if r.match(name):
        masked.append(1)
    else:
        masked.append(0)
        
df_rm["is_top_serie"]=masked

##**Est-ce que le jeu a été lançé par un des studios du top 20 ?**

for studio,i in zip(df["Studio"],range(len(df["Studio"]))):
    if "Ubisoft" in str(studio):
        df=df.replace(to_replace=studio,value="Ubisoft")
        
for studio,i in zip(df["Studio"],range(len(df["Studio"]))):
    if "EA" in str(studio):
        df=df.replace(to_replace=studio,value="EA")
    elif "Electronic Arts" in str(studio):
        df=df.replace(to_replace=studio,value="EA")
        
top_studio=df.groupby("Studio").agg({"Global_Sales":sum}).reset_index().sort_values(by="Global_Sales",ascending=False)
top_studio=top_studio.head(20)

liste_top=top_studio["Studio"]

chaine = '|'.join(liste_top.values)
r=re.compile(chaine)

masked=[]
for studio in df_rm["Studio"]:
    if r.match(str(studio)):
        masked.append(1)
    else:
        masked.append(0)
        
df_rm["is_top_studio"]=masked

##**Est-ce que le jeu a été cité au festival E3 ?**

link_e3_before2009=["https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2000",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2001",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2002",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2003",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2004",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2005",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2006",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2009"]

link_e3_after2009=["https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2010",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2011",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2012",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2013",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2014",
"https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2015"]

link_e3_2016="https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2016"

li_final=[]

for url in link_e3_before2009:
    url_scrap = str(url)
    r = requests.get(url_scrap)
    soup = BeautifulSoup(r.content, 'html.parser')

    div = soup.find_all("div",class_="mw-body-content mw-content-ltr")
    

    for element in div:
        li=element.find_all("i")
                
        for i in li:
            li_final.append(i.text)
            
for url in link_e3_after2009:
    url_scrap = str(url)
    r = requests.get(url_scrap)
    soup = BeautifulSoup(r.content, 'html.parser')

    div = soup.find_all("table",class_="wikitable")
    

    for element in div:
        li=element.find_all("i")
                
        for i in li:
            li_final.append(i.text)
            
url_scrap ="https://fr.wikipedia.org/wiki/Electronic_Entertainment_Expo_2016"
r = requests.get(url_scrap)
soup = BeautifulSoup(r.content, 'html.parser')

div = soup.find_all("table",{"role":True})
    
for element in div:
    li=element.find_all("i")
                
    for i in li:
        li_final.append(i.text)
        
li_final=pd.Series(li_final)

chaine = '|'.join(li_final.values)
r=re.compile(chaine)

masked=[]
for name in df_rm["Name"]:
    if r.match(name):
        masked.append(1)
    else:
        masked.append(0)
        
df_rm["is_e3"]=masked

##**Est-ce que le jeu a été lançé en même temps qu'une console ?**

def extract_games_launch(url):
    
    li_games_launch=[]
    
    names = url.find_all("div",class_="mw-body-content mw-content-ltr")
    
    for element in names:
        li=element.find_all("i")
            
    for i in li:
        if i.text not in li_games_launch:
            li_games_launch.append(i.text)
    
    return li_games_launch
    
url_scrap = "https://fr.wikipedia.org/wiki/Liste_de_jeux_au_lancement_de_consoles_de_jeux_vid%C3%A9o"
url_page = requests.get(url_scrap)
parsed_url_page = BeautifulSoup(url_page.content, 'html.parser')
data_full=extract_games_launch(parsed_url_page)

data_full=pd.Series(data_full)

chaine = '|'.join(data_full.values)
r=re.compile(chaine)

masked=[]
for name in df_rm["Name"]:
    if r.match(name):
        masked.append(1)
    else:
        masked.append(0)
        
df_rm["is_launch_plateform_associated"]=masked

In [159]:
df_rm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16538 entries, 0 to 16537
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Name                            16538 non-null  object        
 1   Platform                        16538 non-null  object        
 2   Year                            16538 non-null  object        
 3   Genre                           16538 non-null  object        
 4   Publisher                       16538 non-null  object        
 5   Studio                          16538 non-null  object        
 6   NA_Sales                        16538 non-null  float64       
 7   EU_Sales                        16538 non-null  float64       
 8   JP_Sales                        16538 non-null  float64       
 9   Other_Sales                     16538 non-null  float64       
 10  Global_Sales                    16538 non-null  float64       
 11  GK

In [160]:
df_rm.head()

,Name,Platform,Year,Genre,Publisher,Studio,NA_Sales,EU_Sales,JP_Sales,Other_Sales,...,RM_Studio_reviews,RM_Licence,RM_Licence_score,RM_Licence_rate,RM_Licence_reviews,is_serie,is_top_serie,is_top_studio,is_e3,is_launch_plateform_associated
0,Wii Sports,Wii,2006,Sports,Nintendo,Nintendo,41.49,29.02,3.77,8.46,...,229.0,30.01,8.45,8.10,229.0,1,1,1,1,1
1,Super Mario Bros.,NES,1985,Platform,Nintendo,Nintendo,29.08,3.58,6.81,0.77,...,12.0,1.59,7.00,7.10,172.0,1,1,1,0,1
2,Mario Kart Wii,Wii,2008,Racing,Nintendo,Nintendo,15.85,12.88,3.79,3.31,...,39.5,9.02,7.10,7.35,28.0,1,1,1,0,0
3,Wii Sports Resort,Wii,2009,Sports,Nintendo,Nintendo,15.75,11.01,3.28,2.96,...,136.0,82.74,7.60,7.05,210.0,1,1,1,1,1
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,Nintendo,11.27,8.89,10.22,1.00,...,197.0,0.06,9.00,8.90,197.0,1,1,1,0,0


In [161]:
df_rm.to_csv('/Users/jeremy/Desktop/DATASCIENTEST/02_Projet_GameSpy/gamespy_best_dataset.csv')

In [162]:
df_rm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16538 entries, 0 to 16537
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Name                            16538 non-null  object        
 1   Platform                        16538 non-null  object        
 2   Year                            16538 non-null  object        
 3   Genre                           16538 non-null  object        
 4   Publisher                       16538 non-null  object        
 5   Studio                          16538 non-null  object        
 6   NA_Sales                        16538 non-null  float64       
 7   EU_Sales                        16538 non-null  float64       
 8   JP_Sales                        16538 non-null  float64       
 9   Other_Sales                     16538 non-null  float64       
 10  Global_Sales                    16538 non-null  float64       
 11  GK

# MODELISATION - CLASSIFICATION

In [163]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

## 4 catégories

In [164]:
df_rm_tosplit = df_rm

In [165]:
df_rm_tosplit['Sales_cat'] = pd.cut(df_rm_tosplit['Global_Sales'], bins=[0,0.1,0.249,1, 100000], 
       labels=[1,2,3,4])
df_rm_tosplit['Sales_cat'] = df_rm_tosplit['Sales_cat'].astype("int")
df_rm_tosplit = df_rm_tosplit.drop(['Name', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Date_Sortie'], axis=1)

cat_cols = df_rm_tosplit.select_dtypes(include=['object']).columns.to_list()
for col in cat_cols:
    i = 1
    occurences = df_rm_tosplit[col].unique()
    for occ in occurences:
        df_rm_tosplit.loc[df_rm_tosplit[col]==occ, col] = i
        i+=1
        
df_rm_tosplit[cat_cols] = df_rm_tosplit[cat_cols].astype('float')
df_rm_tosplit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16538 entries, 0 to 16537
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Platform                        16538 non-null  float64
 1   Year                            16538 non-null  float64
 2   Genre                           16538 non-null  float64
 3   Publisher                       16538 non-null  float64
 4   Studio                          16538 non-null  float64
 5   GK_licence                      16538 non-null  float64
 6   Mois                            16538 non-null  float64
 7   RM_Publisher                    16516 non-null  float64
 8   RM_Publisher_score              14347 non-null  float64
 9   RM_Publisher_rate               14347 non-null  float64
 10  RM_Publisher_reviews            13076 non-null  float64
 11  RM_Studio                       16516 non-null  float64
 12  RM_Studio_score                 

In [166]:
target=df_rm_tosplit["Sales_cat"]
data=df_rm_tosplit.drop("Sales_cat",axis=1)

X_train,X_test,y_train,y_test=train_test_split(data,target,test_size=0.2)

In [167]:
RM_Publisher_mean = X_train['RM_Publisher'].mean()
RM_Publisher_score_mean = X_train['RM_Publisher_score'].mean()
RM_Publisher_rate_mean = X_train['RM_Publisher_rate'].mean()
RM_Publisher_reviews_mean = X_train['RM_Publisher_reviews'].mean()

RM_Studio_mean = X_train['RM_Studio'].mean()
RM_Studio_score_mean = X_train['RM_Studio_score'].mean()
RM_Studio_rate_mean = X_train['RM_Studio_rate'].mean()
RM_Studio_reviews_mean = X_train['RM_Studio_reviews'].mean()

RM_Licence_mean = X_train['RM_Licence'].mean()
RM_Licence_score_mean = X_train['RM_Licence_score'].mean()
RM_Licence_rate_mean = X_train['RM_Licence_rate'].mean()
RM_Licence_reviews_mean = X_train['RM_Licence_reviews'].mean()

In [168]:
X_train['RM_Publisher'] = X_train['RM_Publisher'].fillna(RM_Publisher_mean)
X_train['RM_Publisher_score'] = X_train['RM_Publisher_score'].fillna(RM_Publisher_score_mean)
X_train['RM_Publisher_rate'] = X_train['RM_Publisher_rate'].fillna(RM_Publisher_rate_mean)
X_train['RM_Publisher_reviews'] = X_train['RM_Publisher_reviews'].fillna(RM_Publisher_reviews_mean)

X_train['RM_Studio'] = X_train['RM_Studio'].fillna(RM_Studio_mean)
X_train['RM_Studio_score'] = X_train['RM_Studio_score'].fillna(RM_Studio_score_mean)
X_train['RM_Studio_rate'] = X_train['RM_Studio_rate'].fillna(RM_Studio_rate_mean)
X_train['RM_Studio_reviews'] = X_train['RM_Studio_reviews'].fillna(RM_Studio_reviews_mean)

X_train['RM_Licence'] = X_train['RM_Licence'].fillna(RM_Licence_mean)
X_train['RM_Licence_score'] = X_train['RM_Licence_score'].fillna(RM_Licence_score_mean)
X_train['RM_Licence_rate'] = X_train['RM_Licence_rate'].fillna(RM_Licence_rate_mean)
X_train['RM_Licence_reviews'] = X_train['RM_Licence_reviews'].fillna(RM_Licence_reviews_mean)

In [169]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13230 entries, 7421 to 13866
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Platform                        13230 non-null  float64
 1   Year                            13230 non-null  float64
 2   Genre                           13230 non-null  float64
 3   Publisher                       13230 non-null  float64
 4   Studio                          13230 non-null  float64
 5   GK_licence                      13230 non-null  float64
 6   Mois                            13230 non-null  float64
 7   RM_Publisher                    13230 non-null  float64
 8   RM_Publisher_score              13230 non-null  float64
 9   RM_Publisher_rate               13230 non-null  float64
 10  RM_Publisher_reviews            13230 non-null  float64
 11  RM_Studio                       13230 non-null  float64
 12  RM_Studio_score              

In [170]:
X_test['RM_Publisher'] = X_test['RM_Publisher'].fillna(RM_Publisher_mean)
X_test['RM_Publisher_score'] = X_test['RM_Publisher_score'].fillna(RM_Publisher_score_mean)
X_test['RM_Publisher_rate'] = X_test['RM_Publisher_rate'].fillna(RM_Publisher_rate_mean)
X_test['RM_Publisher_reviews'] = X_test['RM_Publisher_reviews'].fillna(RM_Publisher_reviews_mean)

X_test['RM_Studio'] = X_test['RM_Studio'].fillna(RM_Studio_mean)
X_test['RM_Studio_score'] = X_test['RM_Studio_score'].fillna(RM_Studio_score_mean)
X_test['RM_Studio_rate'] = X_test['RM_Studio_rate'].fillna(RM_Studio_rate_mean)
X_test['RM_Studio_reviews'] = X_test['RM_Studio_reviews'].fillna(RM_Studio_reviews_mean)

X_test['RM_Licence'] = X_test['RM_Licence'].fillna(RM_Licence_mean)
X_test['RM_Licence_score'] = X_test['RM_Licence_score'].fillna(RM_Licence_score_mean)
X_test['RM_Licence_rate'] = X_test['RM_Licence_rate'].fillna(RM_Licence_rate_mean)
X_test['RM_Licence_reviews'] = X_test['RM_Licence_reviews'].fillna(RM_Licence_reviews_mean)

In [171]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3308 entries, 7709 to 12654
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Platform                        3308 non-null   float64
 1   Year                            3308 non-null   float64
 2   Genre                           3308 non-null   float64
 3   Publisher                       3308 non-null   float64
 4   Studio                          3308 non-null   float64
 5   GK_licence                      3308 non-null   float64
 6   Mois                            3308 non-null   float64
 7   RM_Publisher                    3308 non-null   float64
 8   RM_Publisher_score              3308 non-null   float64
 9   RM_Publisher_rate               3308 non-null   float64
 10  RM_Publisher_reviews            3308 non-null   float64
 11  RM_Studio                       3308 non-null   float64
 12  RM_Studio_score               

In [172]:
y_train.value_counts()

1    4861
3    3795
2    2917
4    1657
Name: Sales_cat, dtype: int64

In [173]:
def compute_score(y,pred):
    return np.abs(pred - y).sum()/(3*y.size)

In [174]:
knn=KNeighborsClassifier()
knn.fit(X_train,y_train)
y_pred_test=knn.predict(X_test)
metric_eval_knn=compute_score(y_test,y_pred_test)
print(" KNN compute score :" , metric_eval_knn)
print(" KNN score :" , knn.score(X_test,y_test))

rfc=RandomForestClassifier()
rfc.fit(X_train,y_train)
y_pred_test=rfc.predict(X_test)
metric_eval_rfc=compute_score(y_test,y_pred_test)
print(" RF compute score" , metric_eval_rfc)
print(" RF score :" , rfc.score(X_test,y_test))

dt=DecisionTreeClassifier()
dt.fit(X_train,y_train)
y_pred_test=dt.predict(X_test)
metric_eval_dt=compute_score(y_test,y_pred_test)
print(" DT compute score" , metric_eval_dt)
print(" DT score :" , dt.score(X_test,y_test))

lg=LogisticRegression(max_iter=1000,class_weight="balanced")
lg.fit(X_train,y_train)
y_pred_test=lg.predict(X_test)
metric_eval_lg=compute_score(y_test,y_pred_test)
print(" LG compute score" , metric_eval_lg)
print(" LG score :" , lg.score(X_test,y_test))

 KNN compute score : 0.18500604594921402
 KNN score : 0.5958282950423216
 RF compute score 0.12162434502216848
 RF score : 0.7255139056831923
 DT compute score 0.16575977428456268
 DT score : 0.6396614268440145
 LG compute score 0.1852075775896816
 LG score : 0.5601571946795647


In [175]:
param_rf=[{'criterion': ['gini', 'entropy'],
           'max_depth': [4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150],
           'max_features': ['auto', 'sqrt', 'log2']}]

gridcv=GridSearchCV(rfc,param_grid=param_rf,cv=3,scoring=make_scorer(compute_score,greater_is_better=False))
gridcv.fit(X_train,y_train)

gridcv.best_params_



{'criterion': 'gini', 'max_depth': 70, 'max_features': 'sqrt'}

In [176]:
rfc_grid = RandomForestClassifier(
    criterion=gridcv.best_params_['criterion'],
    max_depth=gridcv.best_params_['max_depth'],
    max_features=gridcv.best_params_['max_features'])

rfc_grid.fit(X_train, y_train)
y_pred = rfc_grid.predict(X_test)

In [177]:
metric_eval_rfc_grid = compute_score(y_test,y_pred)
print(" RF compute score" , metric_eval_rfc_grid)
print(" RF_grid score :" , rfc_grid.score(X_test,y_test))

 RF compute score 0.11930673115679162
 RF_grid score : 0.7258162031438936


In [178]:
cm = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
cm

Classe prédite,1,2,3,4
Classe réelle,,,,
1,1004,116,145,21
2,81,420,209,11
3,48,52,736,68
4,13,5,138,241


In [179]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.88      0.78      0.83      1286
           2       0.71      0.58      0.64       721
           3       0.60      0.81      0.69       904
           4       0.71      0.61      0.65       397

    accuracy                           0.73      3308
   macro avg       0.72      0.70      0.70      3308
weighted avg       0.74      0.73      0.73      3308



In [180]:
pred4 = pd.concat([X_test, y_test], axis=1)
pred4['y_pred'] = y_pred
pred4['ecart'] = abs(pred4['Sales_cat']-pred4['y_pred'])
pred4

,Platform,Year,Genre,Publisher,Studio,GK_licence,Mois,RM_Publisher,RM_Publisher_score,RM_Publisher_rate,...,RM_Licence_rate,RM_Licence_reviews,is_serie,is_top_serie,is_top_studio,is_e3,is_launch_plateform_associated,Sales_cat,y_pred,ecart
7709,7.0,3.0,4.0,254.0,901.0,2612.0,9.0,0.07,6.900000,7.55000,...,7.550000,18.000000,1,0,0,0,0,2,2,0
13241,4.0,3.0,9.0,15.0,2714.0,4757.0,2.0,0.05,6.150731,6.51125,...,6.655093,47.680694,0,0,0,0,0,1,1,0
34,6.0,21.0,7.0,5.0,11.0,18.0,1.0,0.28,8.000000,6.82000,...,6.820000,330.000000,1,0,1,1,1,4,4,0
10775,4.0,11.0,8.0,6.0,94.0,3726.0,6.0,0.05,7.000000,6.90000,...,7.400000,23.000000,0,0,0,0,0,1,1,0
15022,1.0,11.0,5.0,364.0,2340.0,4071.0,3.0,0.03,6.150731,6.51125,...,6.655093,47.680694,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15795,4.0,11.0,6.0,576.0,3551.0,5951.0,2.0,0.02,6.150731,6.51125,...,6.655093,47.680694,0,0,0,0,0,1,1,0
14068,6.0,12.0,11.0,232.0,676.0,4794.0,12.0,0.29,7.900000,7.90000,...,6.150000,1.000000,1,0,0,0,0,1,1,0
1042,13.0,19.0,2.0,5.0,60.0,418.0,1.0,0.63,8.500000,6.05000,...,6.050000,3.000000,0,0,1,0,0,4,3,1
7382,7.0,16.0,4.0,29.0,157.0,1367.0,8.0,0.26,5.500000,5.50000,...,6.720000,12.000000,0,0,0,0,1,2,3,1


In [181]:
pred4.to_csv('/Users/jeremy/Desktop/DATASCIENTEST/02_Projet_GameSpy/pred_4cat.csv')

# 6 catégories

In [182]:
df_rm_tosplit2 = df_rm

In [183]:
df_rm_tosplit2['Sales_cat'] = pd.cut(df_rm_tosplit2['Global_Sales'], bins=[0,0.1,0.249,0.499, 0.999, 5, 100000], 
       labels=[1,2,3,4,5,6])
df_rm_tosplit2['Sales_cat'] = df_rm_tosplit2['Sales_cat'].astype("int")
df_rm_tosplit2 = df_rm_tosplit2.drop(['Name', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales', 'Date_Sortie'], axis=1)

cat_cols = df_rm_tosplit2.select_dtypes(include=['object']).columns.to_list()
for col in cat_cols:
    i = 1
    occurences = df_rm_tosplit2[col].unique()
    for occ in occurences:
        df_rm_tosplit2.loc[df_rm_tosplit2[col]==occ, col] = i
        i+=1
        
df_rm_tosplit2[cat_cols] = df_rm_tosplit2[cat_cols].astype('float')
df_rm_tosplit2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16538 entries, 0 to 16537
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Platform                        16538 non-null  float64
 1   Year                            16538 non-null  float64
 2   Genre                           16538 non-null  float64
 3   Publisher                       16538 non-null  float64
 4   Studio                          16538 non-null  float64
 5   GK_licence                      16538 non-null  float64
 6   Mois                            16538 non-null  float64
 7   RM_Publisher                    16516 non-null  float64
 8   RM_Publisher_score              14347 non-null  float64
 9   RM_Publisher_rate               14347 non-null  float64
 10  RM_Publisher_reviews            13076 non-null  float64
 11  RM_Studio                       16516 non-null  float64
 12  RM_Studio_score                 

In [184]:
target=df_rm_tosplit2["Sales_cat"]
data=df_rm_tosplit2.drop("Sales_cat",axis=1)

X_train,X_test,y_train,y_test=train_test_split(data,target,test_size=0.2)

In [185]:
RM_Publisher_mean = X_train['RM_Publisher'].mean()
RM_Publisher_score_mean = X_train['RM_Publisher_score'].mean()
RM_Publisher_rate_mean = X_train['RM_Publisher_rate'].mean()
RM_Publisher_reviews_mean = X_train['RM_Publisher_reviews'].mean()

RM_Studio_mean = X_train['RM_Studio'].mean()
RM_Studio_score_mean = X_train['RM_Studio_score'].mean()
RM_Studio_rate_mean = X_train['RM_Studio_rate'].mean()
RM_Studio_reviews_mean = X_train['RM_Studio_reviews'].mean()

RM_Licence_mean = X_train['RM_Licence'].mean()
RM_Licence_score_mean = X_train['RM_Licence_score'].mean()
RM_Licence_rate_mean = X_train['RM_Licence_rate'].mean()
RM_Licence_reviews_mean = X_train['RM_Licence_reviews'].mean()

In [186]:
X_train['RM_Publisher'] = X_train['RM_Publisher'].fillna(RM_Publisher_mean)
X_train['RM_Publisher_score'] = X_train['RM_Publisher_score'].fillna(RM_Publisher_score_mean)
X_train['RM_Publisher_rate'] = X_train['RM_Publisher_rate'].fillna(RM_Publisher_rate_mean)
X_train['RM_Publisher_reviews'] = X_train['RM_Publisher_reviews'].fillna(RM_Publisher_reviews_mean)

X_train['RM_Studio'] = X_train['RM_Studio'].fillna(RM_Studio_mean)
X_train['RM_Studio_score'] = X_train['RM_Studio_score'].fillna(RM_Studio_score_mean)
X_train['RM_Studio_rate'] = X_train['RM_Studio_rate'].fillna(RM_Studio_rate_mean)
X_train['RM_Studio_reviews'] = X_train['RM_Studio_reviews'].fillna(RM_Studio_reviews_mean)

X_train['RM_Licence'] = X_train['RM_Licence'].fillna(RM_Licence_mean)
X_train['RM_Licence_score'] = X_train['RM_Licence_score'].fillna(RM_Licence_score_mean)
X_train['RM_Licence_rate'] = X_train['RM_Licence_rate'].fillna(RM_Licence_rate_mean)
X_train['RM_Licence_reviews'] = X_train['RM_Licence_reviews'].fillna(RM_Licence_reviews_mean)

In [187]:
X_test['RM_Publisher'] = X_test['RM_Publisher'].fillna(RM_Publisher_mean)
X_test['RM_Publisher_score'] = X_test['RM_Publisher_score'].fillna(RM_Publisher_score_mean)
X_test['RM_Publisher_rate'] = X_test['RM_Publisher_rate'].fillna(RM_Publisher_rate_mean)
X_test['RM_Publisher_reviews'] = X_test['RM_Publisher_reviews'].fillna(RM_Publisher_reviews_mean)

X_test['RM_Studio'] = X_test['RM_Studio'].fillna(RM_Studio_mean)
X_test['RM_Studio_score'] = X_test['RM_Studio_score'].fillna(RM_Studio_score_mean)
X_test['RM_Studio_rate'] = X_test['RM_Studio_rate'].fillna(RM_Studio_rate_mean)
X_test['RM_Studio_reviews'] = X_test['RM_Studio_reviews'].fillna(RM_Studio_reviews_mean)

X_test['RM_Licence'] = X_test['RM_Licence'].fillna(RM_Licence_mean)
X_test['RM_Licence_score'] = X_test['RM_Licence_score'].fillna(RM_Licence_score_mean)
X_test['RM_Licence_rate'] = X_test['RM_Licence_rate'].fillna(RM_Licence_rate_mean)
X_test['RM_Licence_reviews'] = X_test['RM_Licence_reviews'].fillna(RM_Licence_reviews_mean)

In [188]:
y_train.value_counts()

1    4888
2    2902
3    2205
4    1552
5    1517
6     166
Name: Sales_cat, dtype: int64

In [189]:
def compute_score(y,pred):
    return np.abs(pred - y).sum()/(3*y.size)

In [190]:
knn=KNeighborsClassifier()
knn.fit(X_train,y_train)
y_pred_test=knn.predict(X_test)
metric_eval_knn=compute_score(y_test,y_pred_test)
print(" KNN compute score :" , metric_eval_knn)
print(" KNN score :" , knn.score(X_test,y_test))

rfc=RandomForestClassifier()
rfc.fit(X_train,y_train)
y_pred_test=rfc.predict(X_test)
metric_eval_rfc=compute_score(y_test,y_pred_test)
print(" RF compute score" , metric_eval_rfc)
print(" RF score :" , rfc.score(X_test,y_test))

dt=DecisionTreeClassifier()
dt.fit(X_train,y_train)
y_pred_test=dt.predict(X_test)
metric_eval_dt=compute_score(y_test,y_pred_test)
print(" DT compute score" , metric_eval_dt)
print(" DT score :" , dt.score(X_test,y_test))

lg=LogisticRegression(max_iter=1000,class_weight="balanced")
lg.fit(X_train,y_train)
y_pred_test=lg.predict(X_test)
metric_eval_lg=compute_score(y_test,y_pred_test)
print(" LG compute score" , metric_eval_lg)
print(" LG score :" , lg.score(X_test,y_test))

 KNN compute score : 0.24969770253929868
 KNN score : 0.5417170495767836
 RF compute score 0.1652559451833938
 RF score : 0.6837968561064087
 DT compute score 0.21281741233373638
 DT score : 0.5985489721886336
 LG compute score 0.26088270858524787
 LG score : 0.48458282950423215


In [ ]:
param_rf=[{'criterion': ['gini', 'entropy'],
           'max_depth': [4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150],
           'max_features': ['auto', 'sqrt', 'log2']}]

gridcv=GridSearchCV(rfc,param_grid=param_rf,cv=3,scoring=make_scorer(compute_score,greater_is_better=False))
gridcv.fit(X_train,y_train)

gridcv.best_params_

In [ ]:
rfc_grid = RandomForestClassifier(
    criterion=gridcv.best_params_['criterion'],
    max_depth=gridcv.best_params_['max_depth'],
    max_features=gridcv.best_params_['max_features'])

rfc_grid.fit(X_train, y_train)
y_pred = rfc_grid.predict(X_test)

In [ ]:
metric_eval_rfc_grid = compute_score(y_test,y_pred)
print(" RF compute score" , metric_eval_rfc_grid)
print(" RF_grid score :" , rfc_grid.score(X_test,y_test))

In [ ]:
cm = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
cm

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
pred6 = pd.concat([X_test, y_test], axis=1)
pred6['y_pred'] = y_pred
pred6['ecart'] = abs(pred6['Sales_cat']-pred6['y_pred'])
pred6

In [ ]:
pred6.to_csv('/Users/jeremy/Desktop/DATASCIENTEST/02_Projet_GameSpy/pred_6cat.csv')